In [2]:
import hashlib
import requests

In [3]:
password = '12345Qwert'
salt = 'whatAnAmazingCon'

# MD5 Hashes (Vulnerable)

In [4]:
def get_md5_hash(text: str) -> str:
    md5_hash = hashlib.md5()
    md5_hash.update(text.encode('utf-8'))
    hash_result = md5_hash.hexdigest()

    return hash_result

def get_md5_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text

    return get_md5_hash(salted_text)



In [5]:
hash = get_md5_hash(password)
salted_hash = get_md5_salted_hash(password, salt)

print(f'The md5 hash is: {hash}')
print(f'The salted md5 hash is: {salted_hash}')

The md5 hash is: 18a49c364ba10330d894b11652e65bc2
The salted md5 hash is: ec31aadc6a06fe53ca079564c48596a6


# SHA 256 Hashes

In [6]:
def get_sha256_hash(text: str) -> str:
    return hashlib.sha256(text.encode()).hexdigest()

def get_sha256_salted_hash(text: str, salt: str) -> str:
    salted_text = salt + text
    return get_sha256_hash(salted_text)

In [7]:
hash = get_sha256_hash(password)
salted_hash = get_sha256_salted_hash(password, salt)

print(f'The sha256 hash is: {hash}')
print(f'The salted sha256 hash is: {salted_hash}')

The sha256 hash is: c90142696762be2f4632c18382315ab39e20a41e997cd8a41502aeafcfae49fe
The salted sha256 hash is: b60c0049182cc396182f62cf598c39a4cfe73ce889f31c840014c09cdc1b3bb6


# Certificates Checker


In [8]:
pip install certvalidator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 11.6 MB/s eta 0:00:00


In [9]:
from oscrypto import tls
from certvalidator import CertificateValidator


expired_cert = 'expired.badssl.com'
self_signed = 'self-signed.badssl.com'
bad_dh = 'dh480.badssl.com'

try:
    session = tls.TLSSession(manual_validation=True)
    connection = tls.TLSSocket(expired_cert, 443, session=session)

    validator = CertificateValidator(connection.certificate, connection.intermediates)
    validator.validate_tls(connection.hostname)
except Exception as err:
    print(f"Unexpected {err}")
    print(f" type: {type(err)}")

Unexpected The path could not be validated because the end-entity certificate expired 2015-04-12 23:59:59Z
 type: <class 'certvalidator.errors.PathValidationError'>


# SSL/TLS Scan

In [14]:
pip install sslyze

In [16]:
!python -m sslyze --mozilla_config=modern netcore


 CHECKING CONNECTIVITY TO SERVER(S)
 ----------------------------------

   mozilla.com:443           => 44.236.72.93 


 SCAN RESULTS FOR MOZILLA.COM:443 - 44.236.72.93
 -----------------------------------------------

 * Certificates Information:
       Hostname sent for SNI:             mozilla.com
       Number of certificates detected:   1


     Certificate #0 ( _RSAPublicKey )
       SHA1 Fingerprint:                  ce66062f234b0bb17c622a3dc3643df5cb131b40
       Common Name:                       mozilla.com
       Issuer:                            R3
       Serial Number:                     387756499525565224674514980380320657733409
       Not Before:                        2023-10-14
       Not After:                         2024-01-12
       Public Key Algorithm:              _RSAPublicKey
       Signature Algorithm:               sha256
       Key Size:                          2048
       Exponent:                          65537
       SubjAltName - DNS Names:        

In [10]:
from datetime import datetime
from pathlib import Path
from typing import List

from sslyze import (
    Scanner,
    ServerScanRequest,
    SslyzeOutputAsJson,
    ServerNetworkLocation,
    ScanCommandAttemptStatusEnum,
    ServerScanStatusEnum,
    ServerScanResult,
    ServerScanResultAsJson,
)
from sslyze.errors import ServerHostnameCouldNotBeResolved
from sslyze.scanner.scan_command_attempt import ScanCommandAttempt


def _print_failed_scan_command_attempt(scan_command_attempt: ScanCommandAttempt) -> None:
    print(
        f"\nError when running ssl_2_0_cipher_suites: {scan_command_attempt.error_reason}:\n"
        f"{scan_command_attempt.error_trace}"
    )


def main() -> None:
    print("=> Starting the scans")
    date_scans_started = datetime.utcnow()

    # First create the scan requests for each server that we want to scan
    try:
        all_scan_requests = [
            ServerScanRequest(server_location=ServerNetworkLocation(hostname="cloudflare.com")),
            ServerScanRequest(server_location=ServerNetworkLocation(hostname="google.com")),
        ]
    except ServerHostnameCouldNotBeResolved:
        # Handle bad input ie. invalid hostnames
        print("Error resolving the supplied hostnames")
        return

    # Then queue all the scans
    scanner = Scanner()
    scanner.queue_scans(all_scan_requests)

    # And retrieve and process the results for each server
    all_server_scan_results = []
    for server_scan_result in scanner.get_results():
        all_server_scan_results.append(server_scan_result)
        print(f"\n\n****Results for {server_scan_result.server_location.hostname}****")

        # Were we able to connect to the server and run the scan?
        if server_scan_result.scan_status == ServerScanStatusEnum.ERROR_NO_CONNECTIVITY:
            # No we weren't
            print(
                f"\nError: Could not connect to {server_scan_result.server_location.hostname}:"
                f" {server_scan_result.connectivity_error_trace}"
            )
            continue

        # Since we were able to run the scan, scan_result is populated
        assert server_scan_result.scan_result

        # Process the result of the SSL 2.0 scan command
        ssl2_attempt = server_scan_result.scan_result.ssl_2_0_cipher_suites
        if ssl2_attempt.status == ScanCommandAttemptStatusEnum.ERROR:
            # An error happened when this scan command was run
            _print_failed_scan_command_attempt(ssl2_attempt)
        elif ssl2_attempt.status == ScanCommandAttemptStatusEnum.COMPLETED:
            # This scan command was run successfully
            ssl2_result = ssl2_attempt.result
            assert ssl2_result
            print("\nAccepted cipher suites for SSL 2.0:")
            for accepted_cipher_suite in ssl2_result.accepted_cipher_suites:
                print(f"* {accepted_cipher_suite.cipher_suite.name}")

        # Process the result of the TLS 1.3 scan command
        tls1_3_attempt = server_scan_result.scan_result.tls_1_3_cipher_suites
        if tls1_3_attempt.status == ScanCommandAttemptStatusEnum.ERROR:
            _print_failed_scan_command_attempt(ssl2_attempt)
        elif tls1_3_attempt.status == ScanCommandAttemptStatusEnum.COMPLETED:
            tls1_3_result = tls1_3_attempt.result
            assert tls1_3_result
            print("\nAccepted cipher suites for TLS 1.3:")
            for accepted_cipher_suite in tls1_3_result.accepted_cipher_suites:
                print(f"* {accepted_cipher_suite.cipher_suite.name}")

        # Process the result of the certificate info scan command
        certinfo_attempt = server_scan_result.scan_result.certificate_info
        if certinfo_attempt.status == ScanCommandAttemptStatusEnum.ERROR:
            _print_failed_scan_command_attempt(certinfo_attempt)
        elif certinfo_attempt.status == ScanCommandAttemptStatusEnum.COMPLETED:
            certinfo_result = certinfo_attempt.result
            assert certinfo_result
            print("\nLeaf certificates deployed:")
            for cert_deployment in certinfo_result.certificate_deployments:
                leaf_cert = cert_deployment.received_certificate_chain[0]
                print(
                    f"{leaf_cert.public_key().__class__.__name__}: {leaf_cert.subject.rfc4514_string()}"
                    f" (Serial: {leaf_cert.serial_number})"
                )

        # etc... Other scan command results to process are in server_scan_result.scan_result

    # Lastly, save the all the results to a JSON file
    json_file_out = Path("api_sample_results.json")
    print(f"\n\n=> Saving scan results to {json_file_out}")
    example_json_result_output(json_file_out, all_server_scan_results, date_scans_started, datetime.utcnow())

    # And ensure we are able to parse them
    print(f"\n\n=> Parsing scan results from {json_file_out}")
    example_json_result_parsing(json_file_out)


def example_json_result_output(
    json_file_out: Path,
    all_server_scan_results: List[ServerScanResult],
    date_scans_started: datetime,
    date_scans_completed: datetime,
) -> None:
    json_output = SslyzeOutputAsJson(
        server_scan_results=[ServerScanResultAsJson.from_orm(result) for result in all_server_scan_results],
        invalid_server_strings=[],  # Not needed here - specific to the CLI interface
        date_scans_started=date_scans_started,
        date_scans_completed=date_scans_completed,
    )
    json_output_as_str = json_output.json()  # TODO(#617): Switch to model_dump_json()
    json_file_out.write_text(json_output_as_str)


def example_json_result_parsing(results_as_json_file: Path) -> None:
    # SSLyze scan results serialized to JSON were saved to this file using --json_out
    results_as_json = results_as_json_file.read_text()

    # These results can be parsed
    parsed_results = SslyzeOutputAsJson.parse_raw(results_as_json)

    # Making it easy to do post-processing and inspection of the results
    print("The following servers were scanned:")
    for server_scan_result in parsed_results.server_scan_results:
        print(f"\n****{server_scan_result.server_location.hostname}:{server_scan_result.server_location.port}****")

        if server_scan_result.scan_status == ServerScanStatusEnum.ERROR_NO_CONNECTIVITY:
            print(f"That scan failed with the following error:\n{server_scan_result.connectivity_error_trace}")
            continue

        assert server_scan_result.scan_result
        certinfo_attempt = server_scan_result.scan_result.certificate_info
        if certinfo_attempt.status == ScanCommandAttemptStatusEnum.ERROR:
            _print_failed_scan_command_attempt(certinfo_attempt)  # type: ignore
        else:
            certinfo_result = server_scan_result.scan_result.certificate_info.result
            assert certinfo_result
            for cert_deployment in certinfo_result.certificate_deployments:
                print(f"    SHA1 of leaf certificate: {cert_deployment.received_certificate_chain[0].fingerprint_sha1}")
            print("")

main()

=> Starting the scans


****Results for google.com****

Accepted cipher suites for SSL 2.0:

Accepted cipher suites for TLS 1.3:
* TLS_CHACHA20_POLY1305_SHA256
* TLS_AES_256_GCM_SHA384
* TLS_AES_128_GCM_SHA256

Leaf certificates deployed:
_EllipticCurvePublicKey: CN=*.google.com (Serial: 245575298691591324854831610557896170296)
_RSAPublicKey: CN=*.google.com (Serial: 233574600657540269058516335385323139458)


****Results for cloudflare.com****

Accepted cipher suites for SSL 2.0:

Accepted cipher suites for TLS 1.3:
* TLS_CHACHA20_POLY1305_SHA256
* TLS_AES_256_GCM_SHA384
* TLS_AES_128_GCM_SHA256

Leaf certificates deployed:
_EllipticCurvePublicKey: CN=cloudflare.com,O=Cloudflare\, Inc.,L=San Francisco,ST=California,C=US (Serial: 5233485985088855868343326294380783946)
_RSAPublicKey: CN=cloudflare.com,O=Cloudflare\, Inc.,L=San Francisco,ST=California,C=US (Serial: 20957909947203344368950827704648047993)


=> Saving scan results to api_sample_results.json


=> Parsing scan results from ap

# Check for filtered password using HIBP (Have I Been Pwned?)

In [ ]:
password = '1234'

In [ ]:
base_url = "https://api.pwnedpasswords.com/range"
# Process obtained from https://www.troyhunt.com/ive-just-launched-pwned-passwords-version-2/#cloudflareprivacyandkanonymity
# First we obtain the first 5 characters of the password encoded in SHA-1
sha1_hash = hashlib.sha1(password.encode()).hexdigest()
range = sha1_hash[:5].upper()

# We obtain all filtered passwords in that range and check for ours (with anonimity, there will be about 475 results, we are NOT giving away our password)
r = requests.get(f'{base_url}/{range}')
response_text = r.text
index = response_text.find(sha1_hash[5:].upper())

if index != -1:
  leaked_number = response_text[index:].split('\r\n')[0].split(':')[1]
  print(f'This password has been seen {leaked_number} times before...')
else:
  print('This password has not been leaked (yet...)')


This password has been seen 1473205 times before...


# References
1. [Salt OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#salting)
1. [Pepper OWASP](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html#peppering)
